# Linear data ticket recommender system using reinforcement learning

## Objective

Going to build a data ticket recommender system, which ideally I would want to be able to tell when is a customer going to buy the data ticket furthermore which one will be bought.

## Model

The model is going to be a linear model which is going to learn using stochastic gradient descent with momentum.

In [2]:
import pandas as pd
import numpy as np     
from sklearn.preprocessing import StandardScaler
from datetime import datetime

from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True) 

class LinearModel:
    """ A linear regression model """
    def __init__(self, input_dim, n_action):
        self.W = np.random.randn(input_dim, n_action) / np.sqrt(input_dim)
        self.b = np.zeros(n_action)

        # Momentum terms
        self.vW = 0
        self.vb = 0

        self.losses = []

    def predict(self, X):
        # Make sure X is N x D
        assert(len(X.shape) == 2)
        return X.dot(self.W) + self.b

    def sgd(self, X, Y, learning_rate=0.05, momentum=0.25):
        # Make sure X is N x D
        assert(len(X.shape) == 2)

        # The loss values are 2-D
        # Normally we would divide by N only
        # But now we divide by N x K
        num_values = np.prod(Y.shape)

        # Do one step of gradient descent
        # We multiply by 2 to get the exact gradient
        # (not adjusting the learning rate)
        # i.e. d/dx (x^2) --> 2x
        Yhat = self.predict(X)
        gW = 2 * X.T.dot(Yhat - Y) / num_values
        gb = 2 * (Yhat - Y).sum(axis=0) / num_values

        # Update momentum terms
        self.vW = momentum * self.vW - learning_rate * gW
        self.vb = momentum * self.vb - learning_rate * gb

        # Update params
        self.W += self.vW
        self.b += self.vb

        mse = np.mean((Yhat - Y)**2)
        self.losses.append(mse)

/home/eugenio/anaconda3/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



## Enviroment

First we define a scaler for the states of the enviroment. Than the data ticket enviroment with the states consisting of

* Number of times ticket 1 was bought
* Number of times ticket 2 was bought
* Number of times ticket 3 was bought
* Remaining data on the day
* Days since the last data ticket purchase

In the **init** the basic parameters are defined with a convenience **reset** function to reset the enviroment. The possible states are: No tickets is bought, ticket option 1,2 or 3 is bought. We assume that no combination of these is bought ever and one day only one ticket is purchased.

The **offer** function is implemented. After some experimentation I decided to define my reward function based on the avg_days between data ticket purchases, as if I were to give the same reward for a day when ther was a bought ticket and where was not it would just optimise to never offer anything. Clearly giving a larger value for the ticket offers would have made it better, however let us say if a customer buy's ticket only every 100 day, than the reward accumulated over the 100 days has to compete with the reward given on the single purchase day. Thus I decided to scale the reward of the non offering days (1) with the average days between bought tickets.

**_get_obs** simply returns the current state.

In [3]:
def get_scaler(env):
    # Return scikit-learn scaler object to scale the states
    # Note: you could also populate the replay buffer here

    states = []
    for _ in range(env.n_step):
        action = env.action_list[np.random.choice(env.action_space)]
        state, reward, done = env.offer(action)
        states.append(state)
        if done:
            break

    scaler = StandardScaler()
    scaler.fit(states)
    return scaler


class MultiTicketEnv:
    """
    A 3-ticket offering environment.
    State: vector of size 5
        - # nbr times ticket 1 was bought
        - # nbr times ticket 2 was bought
        - # nbr times ticket 3 was bought
        - remaining data
        - days since last ticket
                        
                        (- money on card could be involved however could be too important)
                        
    Action: categorical variable with 4 possibilities
        - for each ticket, you can:
        - 0 = not offer
        - 1 = offer
    """
    def __init__(self, data_tickets, data_usage, random_state=5893):
        np.random.seed(random_state)
        # Data
        self.data_usage = data_usage
        self.data_tickets = data_tickets
        
        # Instance attributes
        self.cur_step = None
        self.last_ticket = None
        self.bought_tickets = None
        self.last_reward = None
        self.avg_days = None
        
        self.state_dim = 6
        self.n_step = data_usage.shape[0]
    
        # Action permutations
        self.action_list = np.array([[1,0,0,0],
                                     [0,1,0,0],
                                     [0,0,1,0],
                                     [0,0,0,1]])
        
        self.action_space = np.arange(len(self.action_list))
        self.reset()
        
    def offer(self, action, train=True):
        """The agent takes the action among the
        possible tickets. Furthermore calculates
        the reward associated with it."""
        
        # Update price, i.e. go to the next day
        data_change = self.data_usage[self.cur_step+1] - self.data_usage[self.cur_step]
        
        # Days counter since last ticket bought
        # Average day counter between tickets
        if data_change <= 0:
            self.last_ticket += 1
        else:
            if train:
                if sum(self.bought_tickets) == 0:
                    self.bought_tickets = self.last_ticket
                else:
                    self.avg_days = sum(self.bought_tickets)/(sum(self.bought_tickets)+1) * self.avg_days + self.last_ticket / (sum(self.bought_tickets)+1)
            self.last_ticket = 0
        
        # Reward average days number of points if ticket is well suggested
        # Reward 1 point if no ticket offers was correct
        # Else 0 reward
        dist = data_tickets['size']-data_change
        boolean = (dist.abs() == dist.abs().min()).astype(int).values
        
        if np.array_equal(action, boolean) and action[0] != 1:
            self.last_reward = round(self.avg_days) # weight by average days of tickets
        elif np.array_equal(action, boolean) and action[0] == 1:
            self.last_reward = 1
        else:
            self.last_reward = 0
        
        # Choose the bought ticket
        self.bought_tickets[boolean.astype(bool)] += 1
        
        # Next step
        self.cur_step += 1

        # Reward is the increase in porfolio value
        reward = self.last_reward
        
        # Done if we have run out of data
        done = self.cur_step == self.n_step - 1

        return self._get_obs(), reward, done
    

    def reset(self):
        """Resets the enviroment to its original state.
        Note that average days is now an educated guess
        at start, which is than the average after the
        first purchase. The educated guess could be based
        on the average ticket distance in days of the
        entire customer base in a real life scenario."""
        self.cur_step = 0
        self.last_ticket = 0
        self.last_reward = 0
        self.avg_days = np.random.randint(low=15,high=20)
        self.bought_tickets = np.zeros(len(self.data_tickets))

        return self._get_obs()

    def _get_obs(self):
        """Returns current state."""
        obs = np.empty(self.state_dim)
        obs[:len(self.bought_tickets)] = self.bought_tickets
        obs[-2] = self.data_usage[self.cur_step]
        obs[-1] = self.last_ticket
        return obs

## Agent

The agent uses the linear model, with decaying exploration (epsilon) after each round and a step discount rate (gamma).

In [4]:
class DQNAgent(object):
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 1    # discount rate
        self.epsilon = 1.0    # exploration rate
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.model = LinearModel(state_size, action_size)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])    # returns action


    def train(self, state, action, reward, next_state, done):
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.amax(self.model.predict(next_state), axis=1)

        target_full = self.model.predict(state)
        target_full[0, action] = target

        # Run one training step
        self.model.sgd(state, target_full)
        
        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Data

I am going to simulate some simple data for now, however this could be enriched with alternating data tickets.

In [5]:
np.random.seed(9345)

data_tickets = pd.DataFrame({'size':[0,500,1000,3000],
                             'price':[0,900,1500,3500]})

n_data = 10
data_usage = np.concatenate([np.linspace(3500,500,15) for _ in range(n_data)]).ravel().tolist()
data_usage = np.array(data_usage)

data_usage = np.concatenate([np.linspace(3500,500,15+np.random.randint(low=-10,high=10)) for _ in range(n_data)]).ravel().tolist()
data_usage = data_usage - abs(np.random.randint(low=0,high=150,size=len(data_usage)))

fig = go.Figure(data=go.Scatter(x=list(range(len(data_usage))),
                                y=data_usage,
                                hovertemplate='<i>Day</i>: %{x:.d}<br>'+\
                                              '<b>Remaining data</b>: %{y:.1f}',
                                name='Simulated data'))
fig.layout.update(title='Simulated sample data',
                  xaxis=dict(title='Days'),
                  yaxis=dict(title='Data usage in MB'),)
fig.show()

# Running the code on the simulated data

A convenience function is defiend to play one episode. Than the agent playes 7000 episodes, so that epsilon decays to the minimum.

In [6]:
def play_one_episode(agent, env, is_train):
    # note: after transforming states are already 1xD
    state = env.reset()
    state = scaler.transform([state])
    done = False
    
    total_reward = []
    actions = []
    while not done:
        action = agent.act(state)
        if is_train == 'test':
            actions.append(action)
        next_state, reward, done = env.offer(env.action_list[action],False)
        next_state = scaler.transform([next_state])
        if is_train == 'train':
            agent.train(state, action, reward, next_state, done)
        state = next_state
        total_reward.append(reward)
    if is_train == 'test': return actions  
    return sum(total_reward)



num_episodes = 7000

env = MultiTicketEnv(data_tickets,data_usage)

state_size = env.state_dim
action_size = len(env.action_list)

agent = DQNAgent(state_size, action_size)

scaler = get_scaler(env)

for e in range(num_episodes):
    t0 = datetime.now()
    val = play_one_episode(agent, env, 'train')
    dt = datetime.now() - t0
    print(f"episode: {e + 1}/{num_episodes}, episode end value: {val:.2f}, duration: {dt}")

episode: 1/7000, episode end value: 73.00, duration: 0:00:00.067149
episode: 2/7000, episode end value: 100.00, duration: 0:00:00.065289
episode: 3/7000, episode end value: 130.00, duration: 0:00:00.064785
episode: 4/7000, episode end value: 116.00, duration: 0:00:00.064516
episode: 5/7000, episode end value: 110.00, duration: 0:00:00.067694
episode: 6/7000, episode end value: 118.00, duration: 0:00:00.066164
episode: 7/7000, episode end value: 136.00, duration: 0:00:00.064584
episode: 8/7000, episode end value: 168.00, duration: 0:00:00.064403
episode: 9/7000, episode end value: 161.00, duration: 0:00:00.067055
episode: 10/7000, episode end value: 155.00, duration: 0:00:00.065836
episode: 11/7000, episode end value: 110.00, duration: 0:00:00.065675
episode: 12/7000, episode end value: 135.00, duration: 0:00:00.064590
episode: 13/7000, episode end value: 183.00, duration: 0:00:00.067242
episode: 14/7000, episode end value: 164.00, duration: 0:00:00.064972
episode: 15/7000, episode end 

episode: 120/7000, episode end value: 154.00, duration: 0:00:00.066473
episode: 121/7000, episode end value: 145.00, duration: 0:00:00.064999
episode: 122/7000, episode end value: 136.00, duration: 0:00:00.064497
episode: 123/7000, episode end value: 145.00, duration: 0:00:00.064321
episode: 124/7000, episode end value: 136.00, duration: 0:00:00.066257
episode: 125/7000, episode end value: 136.00, duration: 0:00:00.064169
episode: 126/7000, episode end value: 154.00, duration: 0:00:00.064756
episode: 127/7000, episode end value: 145.00, duration: 0:00:00.064246
episode: 128/7000, episode end value: 172.00, duration: 0:00:00.066322
episode: 129/7000, episode end value: 145.00, duration: 0:00:00.064846
episode: 130/7000, episode end value: 136.00, duration: 0:00:00.064462
episode: 131/7000, episode end value: 163.00, duration: 0:00:00.064322
episode: 132/7000, episode end value: 146.00, duration: 0:00:00.066452
episode: 133/7000, episode end value: 136.00, duration: 0:00:00.065349
episod

episode: 237/7000, episode end value: 154.00, duration: 0:00:00.066270
episode: 238/7000, episode end value: 163.00, duration: 0:00:00.065818
episode: 239/7000, episode end value: 136.00, duration: 0:00:00.065533
episode: 240/7000, episode end value: 172.00, duration: 0:00:00.065353
episode: 241/7000, episode end value: 145.00, duration: 0:00:00.067170
episode: 242/7000, episode end value: 172.00, duration: 0:00:00.065739
episode: 243/7000, episode end value: 154.00, duration: 0:00:00.064531
episode: 244/7000, episode end value: 154.00, duration: 0:00:00.064522
episode: 245/7000, episode end value: 172.00, duration: 0:00:00.066091
episode: 246/7000, episode end value: 163.00, duration: 0:00:00.064770
episode: 247/7000, episode end value: 172.00, duration: 0:00:00.064278
episode: 248/7000, episode end value: 163.00, duration: 0:00:00.064420
episode: 249/7000, episode end value: 145.00, duration: 0:00:00.066162
episode: 250/7000, episode end value: 172.00, duration: 0:00:00.065119
episod

episode: 353/7000, episode end value: 145.00, duration: 0:00:00.066263
episode: 354/7000, episode end value: 136.00, duration: 0:00:00.064127
episode: 355/7000, episode end value: 172.00, duration: 0:00:00.064719
episode: 356/7000, episode end value: 172.00, duration: 0:00:00.064153
episode: 357/7000, episode end value: 154.00, duration: 0:00:00.066475
episode: 358/7000, episode end value: 145.00, duration: 0:00:00.064853
episode: 359/7000, episode end value: 136.00, duration: 0:00:00.064471
episode: 360/7000, episode end value: 136.00, duration: 0:00:00.064456
episode: 361/7000, episode end value: 172.00, duration: 0:00:00.066192
episode: 362/7000, episode end value: 156.00, duration: 0:00:00.065780
episode: 363/7000, episode end value: 145.00, duration: 0:00:00.065444
episode: 364/7000, episode end value: 163.00, duration: 0:00:00.064684
episode: 365/7000, episode end value: 145.00, duration: 0:00:00.066336
episode: 366/7000, episode end value: 145.00, duration: 0:00:00.064687
episod

episode: 469/7000, episode end value: 136.00, duration: 0:00:00.066637
episode: 470/7000, episode end value: 154.00, duration: 0:00:00.064819
episode: 471/7000, episode end value: 164.00, duration: 0:00:00.065233
episode: 472/7000, episode end value: 137.00, duration: 0:00:00.064192
episode: 473/7000, episode end value: 154.00, duration: 0:00:00.066659
episode: 474/7000, episode end value: 136.00, duration: 0:00:00.064665
episode: 475/7000, episode end value: 154.00, duration: 0:00:00.064381
episode: 476/7000, episode end value: 145.00, duration: 0:00:00.064629
episode: 477/7000, episode end value: 145.00, duration: 0:00:00.067039
episode: 478/7000, episode end value: 172.00, duration: 0:00:00.067350
episode: 479/7000, episode end value: 154.00, duration: 0:00:00.068238
episode: 480/7000, episode end value: 172.00, duration: 0:00:00.064397
episode: 481/7000, episode end value: 163.00, duration: 0:00:00.066751
episode: 482/7000, episode end value: 163.00, duration: 0:00:00.065952
episod

...

episode: 6478/7000, episode end value: 263.00, duration: 0:00:00.066370
episode: 6479/7000, episode end value: 262.00, duration: 0:00:00.070182
episode: 6480/7000, episode end value: 249.00, duration: 0:00:00.066772
episode: 6481/7000, episode end value: 253.00, duration: 0:00:00.067601
episode: 6482/7000, episode end value: 236.00, duration: 0:00:00.069809
episode: 6483/7000, episode end value: 245.00, duration: 0:00:00.066079
episode: 6484/7000, episode end value: 245.00, duration: 0:00:00.066136
episode: 6485/7000, episode end value: 258.00, duration: 0:00:00.070277
episode: 6486/7000, episode end value: 250.00, duration: 0:00:00.066630
episode: 6487/7000, episode end value: 249.00, duration: 0:00:00.067194
episode: 6488/7000, episode end value: 257.00, duration: 0:00:00.070999
episode: 6489/7000, episode end value: 225.00, duration: 0:00:00.067001
episode: 6490/7000, episode end value: 232.00, duration: 0:00:00.067186
episode: 6491/7000, episode end value: 249.00, duration: 0:00:00

episode: 6592/7000, episode end value: 250.00, duration: 0:00:00.067035
episode: 6593/7000, episode end value: 232.00, duration: 0:00:00.069669
episode: 6594/7000, episode end value: 259.00, duration: 0:00:00.068188
episode: 6595/7000, episode end value: 236.00, duration: 0:00:00.067474
episode: 6596/7000, episode end value: 232.00, duration: 0:00:00.069384
episode: 6597/7000, episode end value: 267.00, duration: 0:00:00.068108
episode: 6598/7000, episode end value: 256.00, duration: 0:00:00.067499
episode: 6599/7000, episode end value: 268.00, duration: 0:00:00.069056
episode: 6600/7000, episode end value: 226.00, duration: 0:00:00.068892
episode: 6601/7000, episode end value: 230.00, duration: 0:00:00.066851
episode: 6602/7000, episode end value: 245.00, duration: 0:00:00.069172
episode: 6603/7000, episode end value: 242.00, duration: 0:00:00.067759
episode: 6604/7000, episode end value: 245.00, duration: 0:00:00.066691
episode: 6605/7000, episode end value: 268.00, duration: 0:00:00

episode: 6707/7000, episode end value: 251.00, duration: 0:00:00.068142
episode: 6708/7000, episode end value: 236.00, duration: 0:00:00.066056
episode: 6709/7000, episode end value: 248.00, duration: 0:00:00.065602
episode: 6710/7000, episode end value: 268.00, duration: 0:00:00.065812
episode: 6711/7000, episode end value: 276.00, duration: 0:00:00.067132
episode: 6712/7000, episode end value: 253.00, duration: 0:00:00.066145
episode: 6713/7000, episode end value: 238.00, duration: 0:00:00.065573
episode: 6714/7000, episode end value: 261.00, duration: 0:00:00.066124
episode: 6715/7000, episode end value: 247.00, duration: 0:00:00.067565
episode: 6716/7000, episode end value: 270.00, duration: 0:00:00.066144
episode: 6717/7000, episode end value: 270.00, duration: 0:00:00.066027
episode: 6718/7000, episode end value: 242.00, duration: 0:00:00.065736
episode: 6719/7000, episode end value: 252.00, duration: 0:00:00.067700
episode: 6720/7000, episode end value: 233.00, duration: 0:00:00

episode: 6822/7000, episode end value: 244.00, duration: 0:00:00.067596
episode: 6823/7000, episode end value: 249.00, duration: 0:00:00.066161
episode: 6824/7000, episode end value: 260.00, duration: 0:00:00.065578
episode: 6825/7000, episode end value: 273.00, duration: 0:00:00.065527
episode: 6826/7000, episode end value: 260.00, duration: 0:00:00.067713
episode: 6827/7000, episode end value: 219.00, duration: 0:00:00.067075
episode: 6828/7000, episode end value: 236.00, duration: 0:00:00.066996
episode: 6829/7000, episode end value: 228.00, duration: 0:00:00.065603
episode: 6830/7000, episode end value: 255.00, duration: 0:00:00.067837
episode: 6831/7000, episode end value: 251.00, duration: 0:00:00.066035
episode: 6832/7000, episode end value: 253.00, duration: 0:00:00.065741
episode: 6833/7000, episode end value: 232.00, duration: 0:00:00.065521
episode: 6834/7000, episode end value: 227.00, duration: 0:00:00.068232
episode: 6835/7000, episode end value: 246.00, duration: 0:00:00

episode: 6936/7000, episode end value: 236.00, duration: 0:00:00.067884
episode: 6937/7000, episode end value: 247.00, duration: 0:00:00.066174
episode: 6938/7000, episode end value: 228.00, duration: 0:00:00.065672
episode: 6939/7000, episode end value: 228.00, duration: 0:00:00.065664
episode: 6940/7000, episode end value: 241.00, duration: 0:00:00.067695
episode: 6941/7000, episode end value: 253.00, duration: 0:00:00.066996
episode: 6942/7000, episode end value: 226.00, duration: 0:00:00.066738
episode: 6943/7000, episode end value: 238.00, duration: 0:00:00.065416
episode: 6944/7000, episode end value: 251.00, duration: 0:00:00.067870
episode: 6945/7000, episode end value: 215.00, duration: 0:00:00.065825
episode: 6946/7000, episode end value: 210.00, duration: 0:00:00.065934
episode: 6947/7000, episode end value: 240.00, duration: 0:00:00.065562
episode: 6948/7000, episode end value: 226.00, duration: 0:00:00.067346
episode: 6949/7000, episode end value: 203.00, duration: 0:00:00

## Results on train data

In [7]:
t0 = datetime.now()
val = np.array(play_one_episode(agent, env, 'test'))

data_dict = dict(zip(list(range(4)),
                     ['No offer']+[f'Data ticket {i+1}' for i in range(3)]))

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=list(range(len(data_usage))),
                         y=data_usage,
                         hovertemplate='<i>Day</i>: %{x:.d}<br>'+\
                                       '<b>Remaining data</b>: %{y:.1f}',
                         name='Simulated data'))

fig.add_trace(go.Scatter(x=list(range(1,len(val)+1)),
                         y=val,
                         hovertemplate='%{text}',
                         text=[data_dict[i] for i in val],
                         name='Offer'),
              secondary_y=True)

fig.layout.update(title='Performance on train data',
                  xaxis=dict(title='Days'),
                  yaxis=dict(title='Data usage in MB'),
                  legend=dict(x=.75, y=1.175))

fig.update_yaxes(title='',
                 tickvals=list(data_dict.keys()),
                 ticktext=list(data_dict.values()),
                 secondary_y=True)

fig.show()

## Next steps

A variety of different data tickets would be interesting in the training data. Furthermore a generated test data would be ideal also.